In [ ]:
# sournce: https://www.kaggle.com/code/sahidvelji/meet-the-grandmasters 
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display, HTML
        
KAGGLE_BASE_URL = 'https://kaggle.com/'
TWITTER_BASE_URL = 'https://twitter.com/'
GITHUB_BASE_URL = 'https://github.com/'
#TOP_N = 10

# modified
import os
def beep():
    os.system('afplay /System/Library/Sounds/Ping.aiff')

import pickle


In [ ]:
def display_html(df, cols=None, num_rows=0):
    """Display columns cols and num_rows rows of the data 
    frame df in HTML.
    """
    
    if num_rows != 0:
        df_to_display = df.head(num_rows)
    else:
        df_to_display = df
    
    df_html = df_to_display.to_html(columns=cols, index=False, na_rep='',
                              escape=False, render_links=True)
    display(HTML(df_html))

In [ ]:
# Users
users = pd.read_csv('./data/Users.csv',
                        parse_dates=['RegisterDate']
                    )
user_ach = pd.read_csv('./data/UserAchievements.csv', 
                        parse_dates=['TierAchievementDate']
                    )

beep() # 

string = f"There are {users.shape[0]} data in users."
display(HTML(string))
string = f"There are {users.shape[0]} data in user_ach."
display(HTML(string))


In [ ]:
gms = (users[users['PerformanceTier'] == 3]
        .copy()
        .reset_index(drop=True)
        .drop(columns=['PerformanceTier'])
      )
string = f"There are currently {gms.shape[0]} Grandmasters."
display(HTML(string))

gms_ach = (user_ach[user_ach['Tier'] == 3]
            .copy()
            .reset_index(drop=True)
            .drop(columns=['Tier','TierAchievementDate','Points','CurrentRanking','HighestRanking','TotalGold','TotalSilver','TotalBronze'])
          )
string = f"There are currently {gms_ach.shape[0]} Grandmasters."
display(HTML(string))




In [ ]:
#print(gms.head(3))
uid = gms_ach.UserId.unique()
print(len(uid))

df_GM = pd.DataFrame(columns= ['id','username','displayname','category'])

kgteam = 0
luid = len(uid)
for ii in range(0,luid):
    id = uid[ii]
    udf = users[users["Id"]==id]
    pt = udf.PerformanceTier.iloc[0]
    un = udf.UserName.iloc[0]
    dn = udf.DisplayName.iloc[0]
    if pt == 5:
        kgteam += 1
    pd_tmp = gms_ach[gms_ach["UserId"]==id]
    df_GM.loc[ii] = [id, un, dn, pd_tmp.AchievementType.iloc[0:len(pd_tmp)].tolist()]
    
print(kgteam)
print(df_GM)
print(df_GM.shape)

In [ ]:
pklfl = './master.pkl'
svfile = './masters.xlsx'
sltime = 5

def fetch_profile(luid,stind,df_GM):
    for ii in range(stind,luid):
        time.sleep(3)
        print(ii+1,'/',luid)
        username = df_GM.loc[ii,'username']
        displayname = df_GM.loc[ii,'displayname']
        profile_url = f'{KAGGLE_BASE_URL}{username}'
        print(profile_url)

        result = requests.get(profile_url)
        src = result.text
        tmp = BeautifulSoup(src, 'html.parser').find_all("div", id="site-body")
        if tmp:
            soup = tmp[0].find("script")
        else:
            continue

        user_info = re.search('Kaggle.State.push\(({.*})', str(soup)).group(1)
        user_dict = json.loads(user_info)

        if 'country' in user_dict:
            df_GM.loc[ii, 'Country'] = user_dict['country']
        if 'city' in user_dict:
            df_GM.loc[ii, 'City'] = user_dict['city']
        if 'region' in user_dict:
            df_GM.loc[ii, 'Region'] = user_dict['region']
        if 'userAvatarUrl' in user_dict:
            avatar_url = user_dict['userAvatarUrl']
        else:
            avatar_url = False
        if 'occupation' in user_dict:
            df_GM.loc[ii, 'Occupation'] = user_dict['occupation']
        if 'organization' in user_dict:
            df_GM.loc[ii, 'Organization'] = user_dict['organization']
        if 'followers' in user_dict:
            if 'count' in user_dict['followers']:
                df_GM.loc[ii, 'NumFollowers'] = user_dict['followers']['count']
        if 'following' in user_dict:
            if 'count' in user_dict['following']:
                df_GM.loc[ii, 'NumFollowing'] = user_dict['following']['count']
        if 'discussionsSummary' in user_dict:
            if 'totalResults' in user_dict['discussionsSummary']:
                df_GM.loc[ii, 'NumPosts'] = user_dict['discussionsSummary']['totalResults']
        if 'datasetSummary' in user_dict:
            if 'totalResults' in user_dict['datasetsSummary']:
                df_GM.loc[ii, 'NumDatasets'] = user_dict['datasetsSummary']['totalResults']
        if 'scriptsSummary' in user_dict:
            if 'totalResults' in user_dict['scriptsSummary']:
                df_GM.loc[ii, 'NumKernels'] = user_dict['scriptsSummary']['totalResults']
        if 'competitionsSummary' in user_dict:
            if 'totalResults' in user_dict['competitionsSummary']:
                df_GM.loc[ii, 'NumCompetitions'] = user_dict['competitionsSummary']['totalResults']
        if 'linkedInUrl' in user_dict:
            linkedIn_url = user_dict['linkedInUrl']
        else:
            linkedIn_url = False
        if 'twitterUserName' in user_dict:
            twitter_username = user_dict['twitterUserName']
        else:
            twitter_username = False
        if 'gitHubUserName' in user_dict:
            github_username = user_dict['gitHubUserName']
        else:
            github_username = False

        print(displayname)
        if profile_url:
            if pd.isnan(displayname):
                if avatar_url:
                    df_GM.loc[ii, 'Image'] = f'<a href="{profile_url}" target="_blank" title="{displayname}"><img src="{avatar_url}" width="100" height="100"></a>'
        if avatar_url and avatar_url != '':
            df_GM.loc[ii,'ImageUrl'] = f'= image("{avatar_url}")'
        if profile_url and profile_url != '':
            df_GM.loc[ii,'ProfileUrl'] = f'= hyperlink("{profile_url}")'
        if twitter_username and twitter_username != '':
            df_GM.loc[ii, 'twitter'] = f'=hyperlink("{TWITTER_BASE_URL}{twitter_username}")'
        if github_username and github_username != '':
            df_GM.loc[ii, 'github'] = f'=hyperlink("{GITHUB_BASE_URL}{github_username}")'
        if linkedIn_url and linkedIn_url != '':
            df_GM.loc[ii, 'linkedin'] = f'=hyperlink("{linkedIn_url}")'
        
        if pd.isna(df_GM.loc[ii,'ProfileUrl']):
            break
        



In [ ]:

if os.path.exists(pklfl):
    #with open(pklfl,'rb') as f:
    #    gms,gms_ach,df_GM = pickle.load(f)

    stind = 0
    for ii in range(0,luid):
        profile_url = df_GM.loc[ii,'ProfileUrl']
        if pd.isna(profile_url):
            stind = ii
            break
        with open(pklfl,'wb') as f:
            pickle.dump([gms,gms_ach,df_GM],f)

    print(stind)

    fetch_profile(luid,stind,df_GM)

else:
    fetch_profile(luid,0,df_GM)
    with open(pklfl,'wb') as f:
        pickle.dump([gms,gms_ach,df_GM],f)

gms = gms.convert_dtypes()
gms_ach = gms_ach.convert_dtypes()
df_GM = df_GM.convert_dtypes()

df_GM.to_excel(svfile, sheet_name='GrandMasters',\
                columns = ['id','username','displayname','category','ImageUrl', 'ProfileUrl',
                           'twitter', 'github', 'linkedin', 'Country', 'Region', 'City', 'Occupation', 'Organization', 'NumFollowers'])

beep() # modified